<a href="https://colab.research.google.com/github/neethanm/Knowledge-Distillation/blob/main/Distillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:

from indic_transliteration import sanscript
from indic_transliteration.sanscript import SchemeMap, SCHEMES, transliterate
import regex as re
import torch
import pandas as pd

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
def transliterate_text(text, from_lang='hk', to_lang='kannada'):
    # hk - harvard kyoto romanization scheme
    return transliterate(text, from_lang, to_lang)

# def is_kannada_script(text):
# # Regular expression to match Kannada characters and variants
#     kannada_pattern = re.compile(r'[\p{Script=Kannada}\p{Script=Kannada_Extensions}\p{Script=Kannada_Supplement}]+', re.UNICODE)
#     return bool(kannada_pattern.search(text))

from sklearn.preprocessing import LabelEncoder

# Encode the sentiment labels
def encode_sentiments(df):
    le = LabelEncoder()
    df['sentiment_encoded'] = le.fit_transform(df['Sentiment'])
    return df, le

In [5]:
df_train = pd.read_csv("gdrive/My Drive/Colab Notebooks/kannada_sentiment_train.csv")
final_train_arr = []

for index, row in df_train.iterrows():
    # Convert the row to a string and then print
    row_str = ', '.join(map(str, row))
    row_str.strip();
    arr = row_str.split(";");
    arr[0] = arr[0].strip()

    arr[1] = arr[1].strip()
    arr[0] = re.sub(r'[^\w\s]','',transliterate_text(arr[0])) # removing punctuation and transliteration
    final_train_arr.append([arr[0], arr[1]])

final_train_df = pd.DataFrame(final_train_arr, columns=['Reviews', 'Sentiment'])
final_train_df, label_encoder = encode_sentiments(final_train_df)

In [6]:
final_train_df['sentiment_encoded'] -= 1
print(final_train_df)
invalid_labels = final_train_df[final_train_df['sentiment_encoded']<0]
print(invalid_labels)
final_train_df = final_train_df[final_train_df['sentiment_encoded']>=0]

                                                Reviews      Sentiment  \
0                   Bರೋ ನಿನ್ ಏಲ್ಲೋ ಹೋಗ್ಬುತ್ತೇ ಬೇದು ಬ್ರೋ       Positive   
1     ಪೃಥ್ವಿ ಅಂಬರ್ ಏನ್  ಗುರೂ ನೀನು ನತಿಓನಲ್ ಅwಅರ್ದ್ wಇ...       Positive   
2     ಸರ್ ಅಶ್ವತ್ಥಾಮ ಆ ವೈದ್ಯರ ಮನೆಗೆ ಬರುತ್ತಿದ್ದ ಅಂತ ಹೇ...  unknown_state   
3                                   ಬೇಸ್ತ್ ಸೋನ್ಗ್ ಏವೇರ್    not-Kannada   
4              ಷ್wಅಲ್ಪ ಜಸ್ತಿ ಬಿಲ್ದುಪ್ ಅಯಿತು ಅನ್ನಿಸುತ್ತೇ       Negative   
...                                                 ...            ...   
6130  ಆಶಿಕ ಸುಮ್ನೆ ನೀವೇ ಮದುವೆ ಆಗಿ ರಕ್ಷಿತ್ ಶೆಟ್ಟಿ ಅವರಿ...       Positive   
6131                                ಣೇಏವು ನಮ್ಮ ಸ್ಪುರ್ಥಿ       Positive   
6132                                      ಷುಪೇರ್ ಮೋವಿಏ        Positive   
6133  ಣನ್ದಿ Pಅರ್ಥಸರಥಿ ಛಕ್ಕ ಅನ್ಸುಥೇ ನಿನು  ಗಅನ್ದು ಸೋಓಲ...       Negative   
6134                          Wಓನ್ದೇರ್ ಫ಼ುಲ್ಲ್ ಸೋನ್ಗ್ಸ್       Positive   

      sentiment_encoded  
0                     2  
1                     2  
2                     4  
3      

In [7]:
df_test = pd.read_csv("gdrive/My Drive/Colab Notebooks/kannada_sentiment_test.csv")
final_test_arr = []

for index, row in df_test.iterrows():
    # Convert the row to a string and then print
    row_str = ', '.join(map(str, row))
    row_str.strip();
    arr = row_str.split(";");
    arr[0] = arr[0].strip()

    arr[1] = arr[1].strip()
    arr[0] = re.sub(r'[^\w\s]','',transliterate_text(arr[0])) # removing punctuation and transliteration
    final_test_arr.append([arr[0], arr[1]])

final_test_df = pd.DataFrame(final_test_arr, columns=['Reviews', 'Sentiment'])
final_test_df, label_encoder = encode_sentiments(final_test_df)

In [8]:
print(final_test_df)

                                               Reviews       Sentiment  \
0                               ೧೦ ಋ ರೇಪೋರ್ತ್ ಮದಿ ಅವ್ನ   unknown_state   
1    ಈ ಚಿತ್ರ ರಿ  ರಿಲೀಸ್  ಆಗ್ಬೇಕು  ಈ ಚಿತ್ರಕ್ಕೆ ಗೌರವ ...        Positive   
2    ಂಅತ್ತೇ ಚೋಮ್ಮೇನ್ತ್ ಯರು ಕೇತ್ತದಗಿ ಮಅದ್ಬರ್ದನ್ಥ ಚೋಮ...        Negative   
3    ನೀನು ಒಳ್ಳೆಯದೇ ಮಾತನಾಡಿದಿಯಾ ಗುರು ನಿನ್ನಗೆ ಧನ್ಯವಾದ...        Positive   
4    ನಮ್ಮ ಕಾಮೆಂಟ್ ನೋಡಿ ಓದಿದವರ ಸಮಸ್ಯೆಗಳು ಬೇಗ ಪರಿಹಾರವ...  Mixed_feelings   
..                                                 ...             ...   
762                                ಯ಼ೇಸ್ ಸೋನ್ಗ್ ಸುಪೇರ್     not-Kannada   
763  ಷೋರ್ರ್ಯ್ ಫ಼ಿಲ್ಮ್ನ  ನಅನು ಥೇಅತೇರ್ ಹೋಗಿ ನೋದ್ಲಿಲ್ಲ...  Mixed_feelings   
764  ಸುದೀಪ್ ಅಣ್ಣನ ಅಭಿಮಾನಿಗಳಿಂದ ಕಡೆಯಿಂದ ಅವನೆ ಶ್ರೀಮನ್...        Positive   
765                            Cಹೇಥನ್ Cಹೇಥು ಜೈ ಡಿ ಬಾಸ್   unknown_state   
766  Eನು Kಅನ್ನದ ಅನ್ಥ ಮತದ್ತರೋ ಏಏ ಗೋಓಬೇಗಲು ಠೋಓ ಟುಮ್ಬ ...        Negative   

     sentiment_encoded  
0                    4  
1                    2  
2                    1  
3          

In [9]:
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification, Trainer, TrainingArguments, DistilBertForSequenceClassification, DataCollatorWithPadding
teacher_model = XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=len(label_encoder.classes_))
student_model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(label_encoder.classes_))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
def tokenize_reviews(df, tokenizer):
    return xlmr_tokenizer(df['Reviews'].tolist(), padding="max_length", truncation=True, return_tensors='pt')

In [11]:
xlmr_tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
distilbert_tokenizer = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
train_encodings = tokenize_reviews(final_train_df, xlmr_tokenizer)
test_encodings = tokenize_reviews(final_test_df, xlmr_tokenizer)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
def validate_token_ids(encodings, max_id):
    for key in encodings:
        encodings[key] = torch.where(encodings[key] > max_id, torch.tensor(max_id), encodings[key])
    return encodings

In [13]:
max_vocab_size = distilbert_tokenizer.config.vocab_size-1
train_encodings = validate_token_ids(train_encodings, max_vocab_size)
test_encodings = validate_token_ids(test_encodings, max_vocab_size)

In [14]:
print(train_encodings)
print(test_encodings)

{'input_ids': tensor([[    0,   335, 21924,  ...,     1,     1,     1],
        [    0, 16125, 30521,  ...,     1,     1,     1],
        [    0, 30521,  3349,  ...,     1,     1,     1],
        ...,
        [    0,     6, 20776,  ...,     1,     1,     1],
        [    0,     6,  6904,  ...,     1,     1,     1],
        [    0,   601, 30521,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}
{'input_ids': tensor([[    0, 30521, 16100,  ...,     1,     1,     1],
        [    0,  2498, 16738,  ...,     1,     1,     1],
        [    0,     6,  6598,  ...,     1,     1,     1],
        ...,
        [    0, 30521, 30521,  ...,     1,     1,     1],
        [    0,   313, 30521,  ...,     1,     1,     1],
        [    0,   241, 20372,  ...,     1,     1,     1]]), 'attentio

In [15]:
sample_review = final_train_df['Reviews'].iloc[0]
tokens = xlmr_tokenizer(sample_review, padding=True, truncation=True, return_tensors='pt')
decoded_text = xlmr_tokenizer.decode(tokens['input_ids'][0], skip_special_tokens=True)

In [31]:
import torch
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels, soft_targets=None):
        self.encodings = encodings
        self.labels = labels
        self.soft_targets = soft_targets

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        if self.soft_targets is not None:
            item['soft_targets'] = torch.tensor(self.soft_targets[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [17]:
train_dataset = SentimentDataset(train_encodings, final_train_df['sentiment_encoded'].tolist())
test_dataset = SentimentDataset(test_encodings, final_test_df['sentiment_encoded'].tolist())

In [18]:
print(train_dataset)
print(test_dataset)

In [19]:
teacher_model = XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=len(label_encoder.classes_))
student_model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=len(label_encoder.classes_))

num_labels = len(label_encoder.classes_)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
from transformers import XLMRobertaForSequenceClassification, Trainer, TrainingArguments

# Initialize the teacher model
teacher_model = XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=num_labels)
teacher_model.gradient_checkpointing_enable()
# Define the training arguments
training_args = TrainingArguments(
    output_dir='./teacher_results',
    fp16 = True,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [21]:
# Initialize the Trainer
trainer = Trainer(
    model=teacher_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [22]:
trainer.train()

<ipython-input-16-bc6ba0d1e941>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss
1,1.438900,1.420255
2,1.429800,1.420195
3,1.363300,1.295565


<ipython-input-16-bc6ba0d1e941>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-16-bc6ba0d1e941>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-16-bc6ba0d1e941>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


TrainOutput(global_step=2301, training_loss=1.4001423729237967, metrics={'train_runtime': 1258.4787, 'train_samples_per_second': 14.622, 'train_steps_per_second': 1.828, 'total_flos': 4841900057622528.0, 'train_loss': 1.4001423729237967, 'epoch': 3.0})

In [32]:
# generating soft targets
import torch
from torch.utils.data import DataLoader

# Create a DataLoader for the training dataset
train_loader = DataLoader(train_dataset, batch_size=8)

# Put the teacher model in evaluation mode
teacher_model.eval()

soft_targets = []

# Generate soft targets
for batch in train_loader:
    inputs = {key: val.to(teacher_model.device) for key, val in batch.items() if key != 'labels'}
    with torch.no_grad():
        outputs = teacher_model(**inputs)
    logits = outputs.logits
    soft_targets.append(torch.nn.functional.softmax(logits, dim=-1))

# Concatenate the soft targets
soft_targets = torch.cat(soft_targets)

<ipython-input-16-bc6ba0d1e941>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


In [33]:
# Custom trainer for Distillation
from transformers import DistilBertForSequenceClassification, DataCollatorWithPadding, Trainer

# Define a custom loss function for knowledge distillation
def distillation_loss(student_logits, teacher_logits, true_labels, alpha=0.5, temperature=2.0):
    # Cross-entropy loss with soft targets
    soft_loss = torch.nn.functional.kl_div(
        input=torch.nn.functional.log_softmax(student_logits / temperature, dim=-1),
        target=torch.nn.functional.softmax(teacher_logits / temperature, dim=-1),
        reduction='batchmean'
    ) * (temperature ** 2)

    # Cross-entropy loss with hard targets
    hard_loss = torch.nn.functional.cross_entropy(student_logits, true_labels)

    # Combine the two losses
    return alpha * soft_loss + (1 - alpha) * hard_loss

# Custom Trainer with distillation loss
class DistillationTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        # Soft targets should be included in the inputs
        teacher_logits = inputs.pop("soft_targets", None)

        # Model output
        outputs = model(**inputs)
        student_logits = outputs.logits

        # Compute distillation loss
        if teacher_logits is not None:
            loss_fct = torch.nn.KLDivLoss(reduction="batchmean")
            loss = loss_fct(
                torch.nn.functional.log_softmax(student_logits, dim=-1),
                torch.nn.functional.softmax(teacher_logits, dim=-1)
            )
        else:
            loss = torch.nn.functional.cross_entropy(student_logits, labels)

        return (loss, outputs) if return_outputs else loss


In [ ]:
# using optuna for hyperparameter tuning
import optuna
from transformers import TrainingArguments
print(soft_targets)

# Define the objective function for Optuna
def objective(trial):
    training_args = TrainingArguments(
        output_dir='./student_results',
        num_train_epochs=trial.suggest_int('num_train_epochs', 2, 4),
        per_device_train_batch_size=trial.suggest_categorical('per_device_train_batch_size', [8, 16, 32]),
        learning_rate=trial.suggest_float('learning_rate', 1e-5, 5e-5, log=True),
        weight_decay=trial.suggest_float('weight_decay', 0.0, 0.3),
        evaluation_strategy="epoch",
        save_strategy="epoch",
        logging_dir='./logs',
        logging_steps=10,
        disable_tqdm=False,
    )

    # Initialize the student model
    student_model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=num_labels)

    # Initialize the student trainer
    student_trainer = DistillationTrainer(
        model=student_model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        data_collator=DataCollatorWithPadding(tokenizer=xlmr_tokenizer),
    )

    # Train the student model
    student_trainer.train()

    # Evaluate the student model
    metrics = student_trainer.evaluate()

    metrics = student_trainer.evaluate()
    print(metrics)

# Create the study and optimize
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=10)

# Print the best hyperparameters
print("Best hyperparameters: ", study.best_params)


[I 2024-07-31 08:55:24,835] A new study created in memory with name: no-name-ad430d59-1b4c-477c-aae5-f51e4b6e59d1


tensor([[0.0908, 0.2125, 0.5558, 0.0581, 0.0828],
        [0.0626, 0.4159, 0.4867, 0.0085, 0.0263],
        [0.0656, 0.3922, 0.5026, 0.0101, 0.0295],
        ...,
        [0.0401, 0.0181, 0.3237, 0.4499, 0.1682],
        [0.0806, 0.2859, 0.5526, 0.0270, 0.0539],
        [0.0404, 0.0182, 0.3265, 0.4450, 0.1699]], device='cuda:0')


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-16-bc6ba0d1e941>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss
1,1.193400,1.217704
2,1.117300,1.178615
3,1.089100,1.151664


<ipython-input-16-bc6ba0d1e941>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-16-bc6ba0d1e941>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-16-bc6ba0d1e941>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-16-bc6ba0d1e941>:8: UserWarning: To copy construct from a 

...

[W 2024-07-31 09:47:45,263] Trial 7 failed with parameters: {'num_train_epochs': 3, 'per_device_train_batch_size': 16, 'learning_rate': 4.5720187305184054e-05, 'weight_decay': 0.04049367433175612} because of the following error: The value None could not be cast to float..
[W 2024-07-31 09:47:45,266] Trial 7 failed with value None.


{'eval_loss': 1.1650036573410034, 'eval_runtime': 4.4708, 'eval_samples_per_second': 171.559, 'eval_steps_per_second': 21.473, 'epoch': 3.0}


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-16-bc6ba0d1e941>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss
1,1.164800,1.226441


<ipython-input-16-bc6ba0d1e941>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


In [ ]:
# training the student model with the best hyperparameters
best_params = study.best_params

training_args = TrainingArguments(
    output_dir='./student_results',
    num_train_epochs=best_params['num_train_epochs'],
    per_device_train_batch_size=best_params['per_device_train_batch_size'],
    learning_rate=best_params['learning_rate'],
    weight_decay=best_params['weight_decay'],
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
    disable_tqdm=False,
)

# Initialize the student model
student_model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=num_labels)

# Include soft targets in the dataset
student_train_dataset = train_dataset.copy()
student_train_dataset["soft_targets"] = soft_targets

# Initialize the student trainer
student_trainer = DistillationTrainer(
    model=student_model,
    args=training_args,
    train_dataset=student_train_dataset,
    eval_dataset=eval_dataset,
    data_collator=DataCollatorWithPadding(tokenizer=xlmr_tokenizer)
)

# Train the student model
student_trainer.train()

# Evaluate the student model
metrics = student_trainer.evaluate()
print("Final evaluation metrics: ", metrics)
